In [1]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2", "3"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses


# GPU memory management
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.75

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
def reconstruction_loss(x, t_decoded):
    '''
    Reconstruction loss for the plain VAE
    '''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def l2_reg_loss(qm):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
#     kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    l2_reg = 0.5*K.square(qm)
    return K.mean(K.sum(l2_reg, axis=-1))


def get_flux_loss(m, state, state_pred):
    '''
    @params:  state, state_pred shape (batch_size, 60, 60, 2)
              p, p_pred shape (batch_size, 60, 60, 1)
              m shape (batch_size, 60, 60, 1)
    
    @return:  loss_flux: scalar
    
    Only consider discrepancies in total flux, not in phases (saturation not used) 
    '''
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.int_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [4]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [5]:
def get_well_bhp_loss(state, state_pred, prod_well_loc):
    '''
    @params: state: shape (batch_size, 60, 60, 2)
             state_pred: shape (batch_size, 60, 60, 2)
             prod_well_loc: shape (5, 2)
             
    p_true: shape (batch_size, 60, 60, 1)
    p_pred: shape (batch_size, 60, 60, 1)
    
    @return: bhp_loss: scalar
    '''
    
    p_true = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)
    
    print(K.int_shape(prod_well_loc))
#     print(K.int_shape(prod_well_loc[0, :, 0]))
#     print(K.int_shape(prod_well_loc[0, :, 1]))
    print(K.int_shape(prod_well_loc[:, 0]))
    print(K.int_shape(prod_well_loc[:, 1]))
    print(prod_well_loc.shape[0])
    bhp_loss = 0
    for i in range(prod_well_loc.shape[0]):
        bhp_loss += K.mean(K.abs(p_true[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :] - p_pred[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :]))
#     bhp_loss = K.sum(p_true[:, prod_well_loc[:, 0], prod_well_loc[:, 1], :] - p_pred[:, prod_well_loc[:,0], prod_well_loc[:,1], :])
    
    return bhp_loss

In [6]:
def create_e2c(latent_dim, u_dim, input_shape, sigma=0.0):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape, sigma=sigma)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)

    return encoder_, decoder_, transition_

In [7]:
# Create plain E2C model and associated loss operations

################### case specification ######################

# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
# data_dir = '/data3/Astro/personal/zjin/datasets/9W_MS_BHP_RATE_GAU/'
data_dir = '/data3/Astro/personal/zjin/datasets/7W_CHA/'

output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

# case_name = '9w_bhp'
# case_name = '9w_bhp_rate'
# case_name = '9w_ms_bhp_rate'
case_name = '7w_cha'

# case_suffix = '_single_out_rel_2'
# case_suffix = '_fix_wl_rel_8'
case_suffix = '_fix_wl_rel_1'

train_suffix = '_with_p'

model_suffix = '_flux_loss'
# model_suffix = '_ae_no_l2_ep_10'
# model_suffix = '_no_fl'


n_train_run = 300
n_eval_run = 100
num_t = 20 
# dt = 200 // num_t
dt = 100
n_train_step = n_train_run * num_t
n_eval_step = n_eval_run * num_t


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat' %(n_train_step, dt, num_t, n_train_run)
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n%d_dt%dday_nt%d_nrun%d.mat' %(n_eval_step, dt, num_t, n_eval_run)

#################### model specification ##################
epoch = 20
batch_size = 4
learning_rate = 1e-4
latent_dim = 50

# u_dim = 9*2 # control dimension, gaussian 9 wells
u_dim = 7*2 # channelized, 7 wells

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
hf_r.close()

num_train = state_t_train.shape[0]
# dt_train = np.ones((num_train,1)) # dt=20days, normalized to 1

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
hf_r.close()

num_eval = state_t_eval.shape[0]
# dt_eval = np.ones((num_eval, 1)) # dt=20days, normalized to 1

In [8]:
# m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
# m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/logk1.dat") # Gaussian
m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case9_cha/template/logk1.dat") # channelized

m = m.reshape(60, 60, 1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))


m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)

m shape is  (60, 60, 1)
m_eval shape is  (2000, 60, 60, 1)
m shape is  (6000, 60, 60, 1)


In [9]:
# well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/well_loc00.dat'
well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case9_cha/template/well_loc00.dat'
well_loc = np.loadtxt(well_loc_file).astype(int)
num_prod = well_loc[0,0]
num_inj = well_loc[0,1]
num_well = num_prod+num_inj
print(num_inj, num_prod)

prod_loc = well_loc[1:num_prod+1,:]
print("prod_loc:\n{}".format(prod_loc))
# inj_loc = well_loc[num_prod+1:,:]
# print("inj_loc:\n{}".format(inj_loc))

print(prod_loc.shape)

print('prod_loc shape is ', prod_loc.shape)
# prod_loc_tf = Input(shape=(5, 2), dtype ='int32')
prod_loc_tf = tf.placeholder(tf.int32, shape=(num_prod,2))

# prod_loc_eval = np.repeat(np.expand_dims(prod_loc, axis = 0), state_t_eval.shape[0], axis = 0)
# print("prod_loc_eval shape is ", prod_loc_eval.shape)
# prod_loc = np.repeat(np.expand_dims(prod_loc,axis = 0), state_t_train.shape[0], axis = 0)
# print("prod_loc shape is ", prod_loc.shape)

3 4
prod_loc:
[[11 10]
 [14 50]
 [40 30]
 [55 20]]
(4, 2)
prod_loc shape is  (4, 2)


In [10]:
# import importlib
# importlib.reload(e2c_util)


In [10]:
# Construct E2C
input_shape = (60, 60, 2)

#############################################
# here we use a UAE framework, sigma = 0.0001
#############################################
encoder, decoder, transition = create_e2c(latent_dim, u_dim, input_shape, sigma=0.0) 


xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))
dt = Input(shape=(1,))

zt = encoder(xt)
xt_rec = decoder(zt)

zt1 = encoder(xt1)

zt1_pred = transition([zt, ut, dt])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1

loss_prod_bhp_t = get_well_bhp_loss(xt, xt_rec, prod_loc_tf) * 20
loss_prod_bhp_t1 = get_well_bhp_loss(xt1, xt1_pred, prod_loc_tf) * 20

loss_l2_reg = l2_reg_loss(zt)  # log(1.) = 0.
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1
loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # JCP 2019 Gaussian case
# loss_bound = loss_rec_t + loss_rec_t1  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # UAE
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg # no flux/bhp loss comparison



# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = l2_reg_loss(zt1_pred - zt1)
# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans

(4, 2)
(4,)
(4,)
4
(4, 2)
(4,)
(4,)
4


In [ ]:
## log for tensorboard
def write_summary(value, tag, summary_writer, global_step):
    """Write a single summary value to tensorboard"""
    summary = tf.Summary()
    summary.value.add(tag=tag, simple_value=value)
    summary_writer.add_summary(summary, global_step)

## used to generate log directory
currentDT = datetime.datetime.now()
current_time = str(currentDT).replace(" ", "-")[:-10]
print(current_time)

summary_writer = tf.summary.FileWriter('logs/' + case_name + case_suffix + '_ep' + str(epoch) + '_tr' + str(n_train_run) + '_' + current_time)

In [11]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

# iterate = K.function([xt, ut, xt1, m_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)
iterate = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, loss_prod_bhp_t, loss_prod_bhp_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch  = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch      = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch        = m[ind0:ind0 + batch_size, ...]
        dt_batch       = dt_train[ind0:ind0 + batch_size, ...]
        
#         print(state_t_batch.shape)
#         print(state_t1_batch.shape)
#         print(bhp_batch.shape)
#         print(m_batch.shape)
#         print(dt_batch.shape)
        

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch, prod_loc, dt_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned
        
        n_itr = e * num_train + ib * batch_size + batch_size
        write_summary(output[0], 'train/total_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[1]+output[2], 'train/sum_rec_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[5]+output[6], 'train/sum_flux_loss', summary_writer, n_itr) # log for tensorboard
        write_summary(output[7]+output[8], 'train/sum_well_loss', summary_writer, n_itr) # log for tensorboard

        if ib % 50 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, prod bhp loss %f, prod bhp loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
            eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval, prod_loc, dt_eval])
            write_summary(eval_loss_val[0], 'eval/total_loss', summary_writer, n_itr) # log for tensorboard
    
    print('====================================================')
    print('\n')
    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))
    print('\n')
    print('====================================================')

encoder.save_weights(output_dir + 'e2c_encoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/20, Batch 1/1500, Loss 5526.879883, Loss rec 2675.247803, loss rec t1 2796.607422, loss kl 0.050211, loss_trans 0.060892, loss flux 3.702467, loss flux t1 2.659565, prod bhp loss 24.090815, prod bhp loss t1 24.460201
Epoch 1/20, Batch 11/1500, Loss 1125.329468, Loss rec 309.584839, loss rec t1 580.738953, loss kl 21.560497, loss_trans 36.938957, loss flux 79.438553, loss flux t1 83.064255, prod bhp loss 4.120461, prod bhp loss t1 9.882990
Epoch 1/20, Batch 21/1500, Loss 598.920593, Loss rec 205.709625, loss rec t1 285.545471, loss kl 9.395497, loss_trans 12.285263, loss flux 44.884441, loss flux t1 35.229576, prod bhp loss 2.819559, prod bhp loss t1 3.051156
Epoch 1/20, Batch 31/1500, Loss 553.407959, Loss rec 236.960846, loss rec t1 205.883179, loss kl 9.234766, loss_trans 10.972992, loss flux 42.612152, loss flux t1 42.765064, prod bhp loss 2.669159, prod bhp loss t1 2.309725
Epoch 1/20, Batch 41/1500, Loss 344.779663, Loss rec 119.154762, loss rec t1 139.756943, loss kl 6.97

Epoch 1/20, Batch 371/1500, Loss 91.257912, Loss rec 38.854046, loss rec t1 32.155502, loss kl 2.283998, loss_trans 1.006545, loss flux 7.286817, loss flux t1 7.678233, prod bhp loss 1.136140, prod bhp loss t1 0.856625
Epoch 1/20, Batch 381/1500, Loss 86.019463, Loss rec 36.342690, loss rec t1 32.027000, loss kl 1.904814, loss_trans 0.859850, loss flux 6.078740, loss flux t1 6.754669, prod bhp loss 0.784560, prod bhp loss t1 1.267138
Epoch 1/20, Batch 391/1500, Loss 80.579407, Loss rec 28.822248, loss rec t1 30.305424, loss kl 2.105341, loss_trans 1.085687, loss flux 8.116656, loss flux t1 8.128065, prod bhp loss 0.870244, prod bhp loss t1 1.145738
Epoch 1/20, Batch 401/1500, Loss 81.728844, Loss rec 25.214710, loss rec t1 33.498405, loss kl 2.036772, loss_trans 1.021219, loss flux 9.050019, loss flux t1 8.641110, prod bhp loss 0.852918, prod bhp loss t1 1.413689
Epoch 1/20, Batch 411/1500, Loss 74.249962, Loss rec 30.007021, loss rec t1 29.745703, loss kl 1.751935, loss_trans 0.630894

Epoch 1/20, Batch 751/1500, Loss 53.698345, Loss rec 20.134888, loss rec t1 20.461584, loss kl 1.330151, loss_trans 0.386198, loss flux 4.783540, loss flux t1 5.369017, prod bhp loss 0.892717, prod bhp loss t1 0.340253
Epoch 1/20, Batch 761/1500, Loss 45.829662, Loss rec 14.115296, loss rec t1 18.623985, loss kl 1.172478, loss_trans 0.317700, loss flux 5.279781, loss flux t1 5.222617, prod bhp loss 0.459224, prod bhp loss t1 0.638580
Epoch 1/20, Batch 771/1500, Loss 51.630508, Loss rec 18.234665, loss rec t1 19.927446, loss kl 1.315719, loss_trans 0.285317, loss flux 5.190932, loss flux t1 5.216989, prod bhp loss 0.739359, prod bhp loss t1 0.720083
Epoch 1/20, Batch 781/1500, Loss 56.132992, Loss rec 17.060448, loss rec t1 25.682278, loss kl 1.020394, loss_trans 0.368749, loss flux 5.029213, loss flux t1 5.069817, prod bhp loss 0.547824, prod bhp loss t1 1.354270
Epoch 1/20, Batch 791/1500, Loss 65.436493, Loss rec 25.564949, loss rec t1 26.677372, loss kl 1.112295, loss_trans 0.206789

Epoch 1/20, Batch 1131/1500, Loss 43.650482, Loss rec 15.986721, loss rec t1 17.500019, loss kl 1.032092, loss_trans 0.116980, loss flux 4.151102, loss flux t1 4.008096, prod bhp loss 0.357109, prod bhp loss t1 0.498360
Epoch 1/20, Batch 1141/1500, Loss 41.444164, Loss rec 15.600081, loss rec t1 16.311918, loss kl 0.732039, loss_trans 0.140249, loss flux 3.972560, loss flux t1 3.799799, prod bhp loss 0.469317, prod bhp loss t1 0.418197
Epoch 1/20, Batch 1151/1500, Loss 45.308079, Loss rec 17.070881, loss rec t1 16.131758, loss kl 1.018411, loss_trans 0.167264, loss flux 4.495217, loss flux t1 4.849457, prod bhp loss 0.935274, prod bhp loss t1 0.639819
Epoch 1/20, Batch 1161/1500, Loss 39.788090, Loss rec 13.018229, loss rec t1 15.394113, loss kl 0.830015, loss_trans 0.168202, loss flux 4.434433, loss flux t1 4.204517, prod bhp loss 0.736385, prod bhp loss t1 1.002195
Epoch 1/20, Batch 1171/1500, Loss 35.823746, Loss rec 11.826468, loss rec t1 13.864779, loss kl 0.837794, loss_trans 0.1

Epoch 2/20, Batch 11/1500, Loss 28.266977, Loss rec 9.420518, loss rec t1 9.867483, loss kl 0.907994, loss_trans 0.079977, loss flux 3.416755, loss flux t1 3.498261, prod bhp loss 0.572827, prod bhp loss t1 0.503160
Epoch 2/20, Batch 21/1500, Loss 31.214098, Loss rec 10.340779, loss rec t1 12.260120, loss kl 0.654709, loss_trans 0.071600, loss flux 3.365862, loss flux t1 3.375338, prod bhp loss 0.482283, prod bhp loss t1 0.663408
Epoch 2/20, Batch 31/1500, Loss 30.062843, Loss rec 9.928114, loss rec t1 10.963757, loss kl 0.885713, loss_trans 0.079977, loss flux 3.369909, loss flux t1 3.741498, prod bhp loss 0.674188, prod bhp loss t1 0.419686
Epoch 2/20, Batch 41/1500, Loss 27.353718, Loss rec 8.728468, loss rec t1 10.062279, loss kl 0.619312, loss_trans 0.088331, loss flux 3.530292, loss flux t1 3.562320, prod bhp loss 0.260628, prod bhp loss t1 0.502087
Epoch 2/20, Batch 51/1500, Loss 32.023830, Loss rec 11.523470, loss rec t1 12.791660, loss kl 0.648912, loss_trans 0.076283, loss fl

...

Epoch 19/20, Batch 1341/1500, Loss 3.439158, Loss rec 0.637117, loss rec t1 0.995631, loss kl 0.085404, loss_trans 0.002154, loss flux 0.647052, loss flux t1 0.683903, prod bhp loss 0.158789, prod bhp loss t1 0.229108
Epoch 19/20, Batch 1351/1500, Loss 4.419960, Loss rec 1.123882, loss rec t1 1.162443, loss kl 0.087752, loss_trans 0.001483, loss flux 0.796029, loss flux t1 0.792629, prod bhp loss 0.216598, prod bhp loss t1 0.239143
Epoch 19/20, Batch 1361/1500, Loss 4.363189, Loss rec 0.860077, loss rec t1 1.217693, loss kl 0.123581, loss_trans 0.002239, loss flux 0.764295, loss flux t1 0.840690, prod bhp loss 0.215996, prod bhp loss t1 0.338617
Epoch 19/20, Batch 1371/1500, Loss 10.162946, Loss rec 3.840142, loss rec t1 3.280232, loss kl 0.112097, loss_trans 0.003829, loss flux 0.978243, loss flux t1 1.016247, prod bhp loss 0.683365, prod bhp loss t1 0.248790
Epoch 19/20, Batch 1381/1500, Loss 5.429681, Loss rec 1.671964, loss rec t1 1.388777, loss kl 0.060314, loss_trans 0.002921, lo

Epoch 20/20, Batch 211/1500, Loss 4.627741, Loss rec 1.077410, loss rec t1 1.305715, loss kl 0.088626, loss_trans 0.001313, loss flux 0.844208, loss flux t1 0.851883, prod bhp loss 0.170810, prod bhp loss t1 0.287776
Epoch 20/20, Batch 221/1500, Loss 3.743620, Loss rec 0.766408, loss rec t1 0.888165, loss kl 0.062977, loss_trans 0.001732, loss flux 0.875222, loss flux t1 0.915733, prod bhp loss 0.088623, prod bhp loss t1 0.144760
Epoch 20/20, Batch 231/1500, Loss 6.783829, Loss rec 1.823681, loss rec t1 2.049498, loss kl 0.097084, loss_trans 0.001365, loss flux 0.901277, loss flux t1 0.918356, prod bhp loss 0.482075, prod bhp loss t1 0.510492
Epoch 20/20, Batch 241/1500, Loss 4.722216, Loss rec 1.069170, loss rec t1 1.395843, loss kl 0.095561, loss_trans 0.001641, loss flux 0.861887, loss flux t1 0.865731, prod bhp loss 0.130782, prod bhp loss t1 0.301602
Epoch 20/20, Batch 251/1500, Loss 4.749834, Loss rec 1.126771, loss rec t1 1.427723, loss kl 0.092273, loss_trans 0.001953, loss flu

Epoch 20/20, Batch 591/1500, Loss 4.643496, Loss rec 1.323624, loss rec t1 1.160105, loss kl 0.106294, loss_trans 0.002402, loss flux 0.783498, loss flux t1 0.725610, prod bhp loss 0.315412, prod bhp loss t1 0.226551
Epoch 20/20, Batch 601/1500, Loss 4.288658, Loss rec 0.948230, loss rec t1 1.263853, loss kl 0.095977, loss_trans 0.002681, loss flux 0.734517, loss flux t1 0.802892, prod bhp loss 0.197592, prod bhp loss t1 0.242915
Epoch 20/20, Batch 611/1500, Loss 4.068592, Loss rec 0.805199, loss rec t1 1.021795, loss kl 0.114941, loss_trans 0.001936, loss flux 0.707818, loss flux t1 0.754513, prod bhp loss 0.286794, prod bhp loss t1 0.375597
Epoch 20/20, Batch 621/1500, Loss 3.854446, Loss rec 0.728114, loss rec t1 1.006344, loss kl 0.098425, loss_trans 0.001923, loss flux 0.765768, loss flux t1 0.783490, prod bhp loss 0.166982, prod bhp loss t1 0.303401
Epoch 20/20, Batch 631/1500, Loss 6.103034, Loss rec 1.381384, loss rec t1 2.057013, loss kl 0.098948, loss_trans 0.002784, loss flu

Epoch 20/20, Batch 971/1500, Loss 3.376975, Loss rec 0.619083, loss rec t1 0.890306, loss kl 0.107893, loss_trans 0.004046, loss flux 0.743081, loss flux t1 0.776199, prod bhp loss 0.061177, prod bhp loss t1 0.175189
Epoch 20/20, Batch 981/1500, Loss 4.447516, Loss rec 1.121280, loss rec t1 1.169786, loss kl 0.094914, loss_trans 0.002595, loss flux 0.865568, loss flux t1 0.886320, prod bhp loss 0.163894, prod bhp loss t1 0.143160
Epoch 20/20, Batch 991/1500, Loss 4.343493, Loss rec 0.810563, loss rec t1 1.349890, loss kl 0.098425, loss_trans 0.001480, loss flux 0.769346, loss flux t1 0.781093, prod bhp loss 0.149918, prod bhp loss t1 0.382777
Epoch 20/20, Batch 1001/1500, Loss 4.480025, Loss rec 1.252831, loss rec t1 1.288742, loss kl 0.069030, loss_trans 0.001654, loss flux 0.795461, loss flux t1 0.787172, prod bhp loss 0.164902, prod bhp loss t1 0.120232
Epoch 20/20, Batch 1011/1500, Loss 4.477625, Loss rec 1.173640, loss rec t1 1.152971, loss kl 0.126685, loss_trans 0.001259, loss f

Epoch 20/20, Batch 1351/1500, Loss 4.180226, Loss rec 0.941111, loss rec t1 1.239256, loss kl 0.089403, loss_trans 0.001408, loss flux 0.774408, loss flux t1 0.779834, prod bhp loss 0.135956, prod bhp loss t1 0.218849
Epoch 20/20, Batch 1361/1500, Loss 4.100067, Loss rec 0.774838, loss rec t1 1.286892, loss kl 0.121313, loss_trans 0.002074, loss flux 0.760698, loss flux t1 0.828590, prod bhp loss 0.114179, prod bhp loss t1 0.211483
Epoch 20/20, Batch 1371/1500, Loss 5.866360, Loss rec 1.657765, loss rec t1 1.654629, loss kl 0.114681, loss_trans 0.003468, loss flux 1.030381, loss flux t1 1.082548, prod bhp loss 0.204879, prod bhp loss t1 0.118009
Epoch 20/20, Batch 1381/1500, Loss 4.643665, Loss rec 1.054400, loss rec t1 1.332811, loss kl 0.059911, loss_trans 0.002837, loss flux 0.830430, loss flux t1 0.829921, prod bhp loss 0.230027, prod bhp loss t1 0.303329
Epoch 20/20, Batch 1391/1500, Loss 6.135566, Loss rec 1.752019, loss rec t1 1.945184, loss kl 0.093318, loss_trans 0.002978, los